<a href="https://colab.research.google.com/github/NinadShegokar/Langchain/blob/main/LlmSherpa.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
!pip install -U langchain scikit-learn langchain_community tiktoken langchain-openai langchainhub chromadb  sentence-transformers   llama-cpp-python
!pip install pypdf dspy-ai fpdf llmsherpa

from IPython.display import clear_output
clear_output()

In [3]:
#imports
import langchain
from torch import cuda, bfloat16
from fpdf import FPDF
import torch
import transformers
from transformers import AutoTokenizer
from time import time
from langchain.chains.summarize import load_summarize_chain
from langchain.llms import HuggingFacePipeline
from langchain.document_loaders import TextLoader,PyPDFLoader, DirectoryLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.embeddings import HuggingFaceEmbeddings
from langchain.chains import MapReduceDocumentsChain, ReduceDocumentsChain,ConversationalRetrievalChain,StuffDocumentsChain
from langchain.text_splitter import CharacterTextSplitter
from langchain.llms import CTransformers
from langchain.vectorstores import FAISS
from langchain.memory import ConversationBufferMemory
from langchain.callbacks.streaming_stdout import StreamingStdOutCallbackHandler
from langchain import PromptTemplate, LLMChain
from langchain_community.llms import LlamaCpp
from langchain_core.callbacks import CallbackManager, StreamingStdOutCallbackHandler
from langchain_core.prompts import PromptTemplate
from langchain.vectorstores import Chroma
from langchain_community.embeddings import HuggingFaceEmbeddings
from langchain_community.vectorstores.utils import DistanceStrategy

In [6]:
from langchain_community.document_loaders.llmsherpa import LLMSherpaFileLoader

loader = LLMSherpaFileLoader(
    file_path="/content/Schizophrenia.pdf",
    new_indent_parser=True,
    apply_ocr=True,
    strategy="text",
    llmsherpa_api_url="https://readers.llmsherpa.com/api/document/developer/parseDocument?renderFormat=all",
)
docs = loader.load()

In [7]:
text_splitter = CharacterTextSplitter.from_tiktoken_encoder(
        chunk_size=1000, chunk_overlap=100
    )
split_docs = text_splitter.split_documents(docs)

In [8]:
!wget "https://huggingface.co/TheBloke/zephyr-7B-beta-GGUF/resolve/main/zephyr-7b-beta.Q4_K_M.gguf"
clear_output()

In [9]:
llm = LlamaCpp(
    model_path="/content/zephyr-7b-beta.Q4_K_M.gguf",
    temperature=0.75,
    max_tokens=1000,
    top_p=1,
    n_ctx=35000,
    verbose=True,  # Verbose is required to pass to the callback manager
)
EMBEDDING_MODEL_NAME = "thenlper/gte-small"

embd = HuggingFaceEmbeddings(
    model_name=EMBEDDING_MODEL_NAME
)
clear_output()

In [10]:
map_prompt = """
Write a concise summary of the following:
"{text}"
CONCISE SUMMARY:
"""
map_prompt_template = PromptTemplate(template=map_prompt, input_variables=["docs"])

In [11]:
combine_prompt = """
Write a concise summary of the following text delimited by triple backquotes.
Return your response in bullet points which covers the key points of the text.
```{text}```
BULLET POINT SUMMARY:
"""
combine_prompt_template = PromptTemplate(template=combine_prompt, input_variables=["docs"])

In [12]:
summary_chain = load_summarize_chain(llm=llm,
                                     chain_type='map_reduce',
                                     map_prompt=map_prompt_template,
                                     combine_prompt=combine_prompt_template,
                                     verbose=True
                                    )

In [13]:
output = summary_chain.run(docs)

/usr/local/lib/python3.10/dist-packages/langchain_core/_api/deprecation.py:117: LangChainDeprecationWarning: The function `run` was deprecated in LangChain 0.1.0 and will be removed in 0.2.0. Use invoke instead.
  warn_deprecated(




> Entering new MapReduceDocumentsChain chain...


> Entering new LLMChain chain...
Prompt after formatting:

Write a concise summary of the following:
"Symptoms
Schizophrenia is characterised by significant impairments in the way reality is perceived and changes in behaviour related to: persistent delusions: the person has fixed beliefs that something is true, despite evidence to the contrary; persistent hallucinations: the person may hear, smell, see, touch, or feel things that are not there; experiences of influence, control or passivity: the experience that one’s feelings, impulses, actions, or thoughts are not generated by oneself, are being placed in one’s mind or withdrawn from one’s mind by others, or that one’s thoughts are being broadcast to others; disorganized thinking, which is often observed as jumbled or irrelevant speech; highly disorganised behaviour e.g.
the person does things that appear bizarre or purposeless, or the person has unpredictable or inappropriate emotio


llama_print_timings:        load time =    4096.18 ms
llama_print_timings:      sample time =     154.33 ms /   236 runs   (    0.65 ms per token,  1529.17 tokens per second)
llama_print_timings: prompt eval time =  544642.21 ms /  1063 tokens (  512.36 ms per token,     1.95 tokens per second)
llama_print_timings:        eval time =  196561.15 ms /   235 runs   (  836.43 ms per token,     1.20 tokens per second)
llama_print_timings:       total time =  742898.55 ms /  1298 tokens
Llama.generate: prefix-match hit



> Finished chain.


> Entering new LLMChain chain...
Prompt after formatting:

Write a concise summary of the following text delimited by triple backquotes.
Return your response in bullet points which covers the key points of the text.
```Schizophrenia is characterized by impairments in perception, delusions, hallucinations, disorganized thinking, behavior, and emotions. It affects approximately 1 in 300 people worldwide, with onset during late adolescence to twenties. People with schizophrenia experience significant distress and impairment in various areas of life. Schizophrenia is not as common as other mental disorders but is associated with higher mortality rates due to physical illnesses. The causes of schizophrenia are unclear, but an interaction between genes and environmental factors is thought to play a role. Most people with schizophrenia do not receive mental health care, and efforts to transfer care from mental hospitals to the community need to be expanded. Quality commun


llama_print_timings:        load time =    4096.18 ms
llama_print_timings:      sample time =     168.69 ms /   255 runs   (    0.66 ms per token,  1511.66 tokens per second)
llama_print_timings: prompt eval time =  137743.93 ms /   278 tokens (  495.48 ms per token,     2.02 tokens per second)
llama_print_timings:        eval time =  203571.80 ms /   254 runs   (  801.46 ms per token,     1.25 tokens per second)
llama_print_timings:       total time =  342830.97 ms /   532 tokens



> Finished chain.

> Finished chain.


In [14]:
print (output)

- Schizophrenia is a mental disorder characterized by impairments in perception, delusions, hallucinations, disorganized thinking, behavior, and emotions.
- It affects approximately 1 in 300 people worldwide with onset during late adolescence to twenties.
- People with schizophrenia experience significant distress and impairment in various areas of life.
- Schizophrenia is not as common as other mental disorders but is associated with higher mortality rates due to physical illnesses.
- The causes of schizophrenia are unclear, but an interaction between genes and environmental factors is thought to play a role.
- Most people with schizophrenia do not receive mental health care.
- Efforts to transfer care from mental hospitals to the community need to be expanded.
- Quality community-based mental health services, including integration in primary health and general hospital care, day centers, supported housing, and outreach services for home-based support, are recommended.
- Stigma, human